In [20]:
import pandas as pd

import numpy as np

import pickle


In [8]:
import requests, json, time

In [9]:
from  tqdm import tqdm_notebook

In [10]:
df_gsc  = pd.read_csv("../data/GSC_PLUS.csv") 

In [11]:
df_gsc.head()

,file,text,HPO_symptoms,HPO_codes,starts,ends
0,1003450,A syndrome of brachydactyly (absence of some m...,"['brachydactyly', 'absence of some middle or d...","['0001156', '0009881', '0001798', '0001792', '...","[14, 29, 74, 86, 105, 120, 166, 210, 232, 346,...","[27, 71, 103, 103, 118, 163, 208, 226, 253, 37..."
1,10051003,Townes-Brocks syndrome (TBS) is an autosomal d...,"['autosomal dominant disorder', 'autosomal dom...","['0000006', '0000006', '0000006', '0003828', '...","[35, 35, 45, 95, 95, 139, 148, 163, 177, 186, ...","[62, 53, 53, 114, 103, 161, 161, 175, 197, 197..."
2,10066029,Nevoid basal cell carcinoma syndrome (NBCCS) i...,"['basal cell carcinoma', 'autosomal dominant',...","['0002671', '0000006', '0000006', '0000006', '...","[7, 89, 89, 99, 99, 143, 143, 208, 238, 250, 2...","[27, 107, 113, 113, 107, 164, 151, 229, 261, 2..."
3,10196695,Angelman syndrome (AS) is a neurodevelopmental...,"['neurodevelopmental disorder', 'contiguous ge...","['0000707', '0001466']","[28, 839]","[55, 863]"
4,10417280,Prader-Willi syndrome (PWS) and Angelman syndr...,"['neurobehavioral disorders', 'de novo']","['0000708', '0003745']","[68, 223]","[93, 230]"


In [12]:
df_gsc.shape

(228, 6)

In [13]:
#TEXTANALYTICS_URL = "http://hacksymptoms-appservice.azurewebsites.net/text/analytics/v3.0-preview.1/domains/health"
TEXTANALYTICS_URL="https://ta4h-app-service.azurewebsites.net/text/analytics/v3.0-preview.1/domains/health"
HEADERS = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
}


In [30]:
BATCH_SIZE = 10
responses = {"documents":[]}
n_in_batch = 1
batch = 1
payload = {"documents":[]}
for i, r in tqdm_notebook(df_gsc.iterrows()):
    doc = {"language":"en", "id":r["file"], "text": r["text"]}
    payload["documents"].append(doc)    
   
    if (n_in_batch == BATCH_SIZE) or (i == (df_gsc.shape[0] - 1)):
        response = requests.request("POST", TEXTANALYTICS_URL, headers=HEADERS, json = payload)
        responses["documents"]+= response.json()["documents"]
        n_in_batch = 1
        payload = {"documents":[]}
        print(response)
        print("processed batch: " + str(batch))
        batch += 1
        
        time.sleep(1)
     
    n_in_batch += 1

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


<Response [200]>
processed batch: 1
<Response [200]>
processed batch: 2
<Response [200]>
processed batch: 3
<Response [200]>
processed batch: 4
<Response [200]>
processed batch: 5
<Response [200]>
processed batch: 6
<Response [200]>
processed batch: 7
<Response [200]>
processed batch: 8
<Response [200]>
processed batch: 9
<Response [200]>
processed batch: 10
<Response [200]>
processed batch: 11
<Response [200]>
processed batch: 12
<Response [200]>
processed batch: 13
<Response [200]>
processed batch: 14
<Response [200]>
processed batch: 15
<Response [200]>
processed batch: 16
<Response [200]>
processed batch: 17
<Response [200]>
processed batch: 18
<Response [200]>
processed batch: 19
<Response [200]>
processed batch: 20
<Response [200]>
processed batch: 21
<Response [200]>
processed batch: 22
<Response [200]>
processed batch: 23
<Response [200]>
processed batch: 24
<Response [200]>
processed batch: 25
<Response [200]>
processed batch: 26



In [34]:
with open('text_analytics_responses.pickle', 'wb') as f:
    pickle.dump(responses, f)

In [35]:
gsc_responses = pd.read_pickle("text_analytics_responses.pickle")

In [36]:
len(gsc_responses["documents"])

228

In [27]:
responses = {"documents":[]}

In [28]:
for r in gsc_responses["documents"]:
    responses["documents"] += r

In [29]:
len(responses["documents"])

226